<a href="https://colab.research.google.com/github/PravinVedurla/Federated-Learning-on-MNIST-data-using-Pysyft-and-Pytorch/blob/master/Federated_Learning_on_MNIST_Data_using_Pysyft_and_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>Installing Syft and then importing Torch with required dependencies</h2>

In [1]:
!pip install syft
!pip install --upgrade --force-reinstall zstd

  Found existing installation: zstd 1.4.0.0
    Uninstalling zstd-1.4.0.0:
      Successfully uninstalled zstd-1.4.0.0


In [0]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms 
import torch.nn.functional as F

torch.set_default_tensor_type(torch.cuda.FloatTensor)  # To tackle an error with pysyft not accepting model transfer to cuda.

## Creating a torch hook to be used along with syft.

In [3]:
import syft as sy
hook = sy.TorchHook(torch)


W0712 15:58:38.994253 139795915089792 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0712 15:58:39.010136 139795915089792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
#creating workers to distribute data to.

bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')
jon = sy.VirtualWorker(hook, id='jon')

In [0]:
#Some parameters defined here

batch_size = 64
test_batch_size = 1000
epochs = 10
log_interval = 30
save_model = False
no_cuda = False

In [6]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


# Loading Data and distributing amongst workers

In [0]:
# defining the transform for both train and test loader.
transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])

federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True, transform=transform).federate((bob, alice, jon)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transform), batch_size=test_batch_size, shuffle=True)

# Building the network itself.

In [0]:
class Classifier(nn.Module):
    """
    Forward Convolutional Neural Network Architecture model
    
    """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [10]:
model = Classifier()
model = model.to(device)  #pushing the model into available device.
optimizer = optim.SGD(model.parameters(), lr=0.01)
for epoch in range(1, epochs + 1):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # iterate through each worker's dataset
        
        model.send(data.location) #send the model to the right location ; data.location returns the worker name in which the data is present
        
        data, target = data.to(device), target.to(device) # pushing both the data and target labels onto the available device.
        
        optimizer.zero_grad() # 1) erase previous gradients (if they exist)
        output = model(data)  # 2) make a prediction
        loss = F.nll_loss(output, target)  # 3) calculate how much we missed
        loss.backward()  # 4) figure out which weights caused us to miss
        optimizer.step()  # 5) change those weights
        model.get()  # get the model back (with gradients)
        
        if batch_idx % log_interval == 0:
            loss = loss.get() #get the loss back
            print('Epoch: {} [Training: {:.0f}%]\tLoss: {:.6f}'.format(epoch, 100. * batch_idx / len(federated_train_loader), loss.item()))
    
    #Testing phase
    
    model.eval()  
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            
            data, target = data.to(device), target.to(device) 
            output = model(data) # Getting a prediction
            
            test_loss += F.nll_loss(output, target, reduction='sum').item() #updating test loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            
            correct += pred.eq(target.view_as(pred)).sum().item() #correct pred in the current test set.

    test_loss /= len(test_loader.dataset) 

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
    
torch.save(model.state_dict(), "mnist_cnn.pt")

Epoch: 1 [Training: 0%]	Loss: 2.318598
Epoch: 1 [Training: 3%]	Loss: 2.211987
Epoch: 1 [Training: 6%]	Loss: 2.058846
Epoch: 1 [Training: 10%]	Loss: 1.601909
Epoch: 1 [Training: 13%]	Loss: 1.053527
Epoch: 1 [Training: 16%]	Loss: 0.682115
Epoch: 1 [Training: 19%]	Loss: 0.708618
Epoch: 1 [Training: 22%]	Loss: 0.464039
Epoch: 1 [Training: 26%]	Loss: 0.417994
Epoch: 1 [Training: 29%]	Loss: 0.473361
Epoch: 1 [Training: 32%]	Loss: 0.442335
Epoch: 1 [Training: 35%]	Loss: 0.357673
Epoch: 1 [Training: 38%]	Loss: 0.247065
Epoch: 1 [Training: 42%]	Loss: 0.254549
Epoch: 1 [Training: 45%]	Loss: 0.479151
Epoch: 1 [Training: 48%]	Loss: 0.207273
Epoch: 1 [Training: 51%]	Loss: 0.194028
Epoch: 1 [Training: 54%]	Loss: 0.233580
Epoch: 1 [Training: 58%]	Loss: 0.214929
Epoch: 1 [Training: 61%]	Loss: 0.274084
Epoch: 1 [Training: 64%]	Loss: 0.213623
Epoch: 1 [Training: 67%]	Loss: 0.165152
Epoch: 1 [Training: 70%]	Loss: 0.241492
Epoch: 1 [Training: 74%]	Loss: 0.335058
Epoch: 1 [Training: 77%]	Loss: 0.171820
Epo

In [11]:
print("Accuracy Obtained {:.4f}%".format( 100. * correct / len(test_loader.dataset)))

Accuracy Obtained 98.9300%
